In [1]:
%matplotlib inline 

import matplotlib
import numpy as np
import matplotlib.pyplot as plt

import math

import pandas as pd
import seaborn as sns

import scipy.integrate as integrate
import scipy.special as sp
import scipy.optimize as spop

import emcee

## Generative model

### Bare model

The model is represented by a 2D Gaussian, with random peak amplitude, width and peak position.

We thus start by defining the 2D and 1D Gaussian function:

In [2]:
def gaussian_2d(x, y, center_x, center_y, amplitude, sigma_x, sigma_y):
    return amplitude*np.exp(-(x-center_x)**2/(2*sigma_x**2))*np.exp(-(y-center_y)/(2*sigma_y**2))

def gaussian_1d(z, center_z, sigma_z, amplitude):
    return amplitude*np.exp(-(z-center_z)**2/(2*sigma_z**2))

We next define the model:

In [3]:
def model(x, y, center_x, center_y, amplitude, sigma_x, sigma_y, background):
    return gaussian_2d(x, y, center_x, center_y, amplitude, sigma_x, sigma_y) + background

### Uncertainties

We consider three possible uncertainty sources:

- a Gaussian statistical uncertainty in the model, $\epsilon_{\mathrm{i}}$;
- a Poissonian uncertainty, given by shot noise in the CCD camera;
- a Gaussian uncertainty with a prior that is proportionally higher if the pixels below it are birther.


## Likelihood function

To construct the likelihood function we need to consider all the uncertainties. Thus, each data point is given by: 

$$y_{\mathrm{i}} = z_{\mathrm{i}} + e_{\mathrm{i}} = m(x_{\mathrm{i}}, \theta) + \epsilon_{\mathrm{i}} + f_{\mathrm{i}} + g_{\mathrm{i}},$$

where $m(x_{\mathrm{i}}, \theta)$ is the model data and $\theta$ the set of parameters that go into the data.

Following the methods of Gregory 4.8, we have the probability distribution for the proposition $Z_i$ that the model we considered gives for the $i$-th data point values in the range $z_{\mathrm{i}}$ to $z_{\mathrm{i}} + dz_{\mathrm{i}}$:

$$p(Z_{\mathrm{i}}|M, \theta, I) = f_{\mathrm{Z}}(z_{\mathrm{i}}) = \frac{1}{\sqrt{2 \pi}\sigma_{\mathrm{mi}}} exp \left( \frac{- \epsilon_i}{2 \sigma_{\mathrm{mi}}} \right)$$.

Note that for simplicity we will consider $\epsilon_{\mathrm{i}} = \epsilon$ and $\sigma_{\mathrm{mi}} = \sigma_{\mathrm{m}}$, for all $i$.

Next the shot noise component is given by:

$$p(F_{\mathrm{i}}|M, \theta, I)  = e^{-\mu_i} \frac{\mu_{\mathrm{i}}^n}{n!}$$

Finally, include the readout noise. For now, only consider it as a simple Gaussian (<font color='red'>TO DO</font>: include prior and change its form):

$$p(G_{\mathrm{i}}|M, \theta, I) = \frac{1}{\sqrt{2 \pi}\sigma_{\mathrm{gi}}} exp \left( \frac{- g_i}{2 \sigma_{\mathrm{gi}}} \right)$$

The likelihood for each pixel is then the convolution of the noise sources and the total likelihood is the product of all likelihoods for each pixel (so for log of the likelihood we perform a sum). For the first stages of the project we perform the convolution directly:

NOTE: In this simple version of the model, the two Gaussian uncertainties convolve to a Gaussian with $sigma = sigma_{\mathrm{m}} + sigma_{\mathrm{g}}$

In [ ]:
def log_likelihood(model_data, sigma_m, sigma_g, mu):
    '''
    sigma_m : uncertainty in the model chosen
    sigma_g : uncertainty from CCD camera readout noise
    mu : average number of photons due to shor noise
    '''
    sigma = sigma_m + sigma_g
    prob_gaussian = gaussian_1d(z, gaussian_2d(x, y, center_x, center_y, amplitude, sigma_x, sigma_y), sigma, 1) 
    prob_poisson
    
    #log_likelihood = np.sum(np.log(np.convolve(prob_gaussian, prob_poisson)))